$$
\newcommand{\F}{\mathbb{F}}
\newcommand{\R}{\mathbb{R}}
\newcommand{\A}{\mathbf{A}}
\newcommand{\L}{\mathbf{L}}
\newcommand{\X}{\mathbf{X}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\v}{\mathbf{v}}
\newcommand{\a}{\mathbf{a}}
\newcommand{\b}{\mathbf{b}}
\newcommand{\c}{\mathbf{c}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\u}{\mathbf{u}}
\newcommand{\0}{\mathbf{0}}
\newcommand{\1}{\mathbf{1}}
$$

## Grad-CAM, an Introduction

We will be discussing the paper written by R.R Selvaraju et al on **[Grad-CAM: Visual Explanations from Deep Networks via Gradient-Based Localization](https://ieeexplore.ieee.org/document/8237336)** on how we can interpret large scale Deep Neural Networks (CNN).

## Terminologies

### Feature Maps

[Feature Maps](https://stats.stackexchange.com/questions/291820/what-is-the-definition-of-a-feature-map-aka-activation-map-in-a-convolutio) has an [intuitive interpretation](https://www.quora.com/What-is-meant-by-feature-maps-in-convolutional-neural-networks), understanding it intuitively should be prioritized than knowing the underlying mathematical structure.

Imagine a 32 by 32 image, consider that a convolutional layer applied on the input and we assume only 2 kernels are applied, then only 2 feature maps are obtained:

- The first feature map is a color filter that is responsible to detect color contrast of an image;
- The second feature map is a [sobel's filter](https://en.wikipedia.org/wiki/Sobel_operator) where it detects edges.

Then assume kernel has size 5 by 5, when convolved on the 32 by 32 image, yields a feature map output of size 28 by 28, although "downsampled", this feature map should still capture valueble spatial information of the 32 by 32 image, and in this case, all edges of the input image (say a cat) will be shown in the feature map 1, we call it $\A^1$, and the color contrast is displayed on feature map 2, $\A^2$.

So the full settings is:

- An input image $\mathcal{X}$ of size $(3, 32, 32)$.
- A CNN but we will only look at its very first layer, a conv layer that has:
    - 2 kernels of size 5 by 5 and default paddings and stride such that the output feature map is 28 by 28.

<img src="https://storage.googleapis.com/reighns/reighns_ml_projects/docs/deep_learning/computer_vision/gradcam_and_variants/cat.PNG" style="margin-left:auto; margin-right:auto"/>
<p style="text-align: center">
    <b>Original Cat Image</b>
</p>

<img src="https://storage.googleapis.com/reighns/reighns_ml_projects/docs/deep_learning/computer_vision/gradcam_and_variants/edge_detectors.PNG" style="margin-left:auto; margin-right:auto"/>
<p style="text-align: center">
    <b>Feature Map A1 and A2; By Hongnan G.</b>
</p>

Notice in the 2 feature maps above, after we pass in the RGB cat image of 3 channels to the first conv layer, we will receive two outputs, called **feature maps**, each of them will describe the image in one way or another. One thing to note is that earlier conv layers will retain a lot detailed information of the image such as the edges, color contrast, but as we go on later, the feature maps become more abstract as they capture high level and abstract details that we human cannot comprehend easily.

### Global Average Pooling (GAP)

The GAP is usually applied after the last conv layer. But for the sake of explanation, we will assume that our "last conv layer" is our first conv layer.

Instead of down sampling patches of the input feature map, global pooling down samples the entire feature map to a single value.

In the last few years, experts have turned to global average pooling (GAP) layers to minimize overfitting by reducing the total number of parameters in the model. Similar to max pooling layers, GAP layers are used to reduce the spatial dimensions of a three-dimensional tensor. However, GAP layers perform a more extreme type of dimensionality reduction, where a tensor with dimensions h×w×d is reduced in size to have dimensions 1×1×d. GAP layers reduce each h×w feature map to a single number by simply taking the average of all hw values.

<img src="https://storage.googleapis.com/reighns/reighns_ml_projects/docs/deep_learning/computer_vision/gradcam_and_variants/gap.png" width="500" height="500" style="margin-left:auto; margin-right:auto"/>
<p style="text-align: center">
    <b>Visualize GAP; Courtesy of Alexis Cook</b>
</p>

Eventually our task is to classify whether the image is a cat or something else. We are unable to succinctly pass these 2 feature maps directly to a linear classifier to classify the cats (i.e. passing in two 28 by 28 kernels to a linear layer does not work but we need linear layer to give us an output). Therefore, we need to flatten these feature maps generated by the conv layers. One way to do this is **Global Average Pooling**.

**A wrong example**

First, I show you a source of confusion on what some perceive GAP as:

For example:

$$
\A^1 = \begin{bmatrix} 2 & 3 \\ 1 & 5 \end{bmatrix} \quad \A^2 = \begin{bmatrix} 3 & 1 \\ 2 & 8 \end{bmatrix}
$$

Assume for a moment $\A^1$ and $\A^2$ are our feature maps (although I mentioned it was 28 by 28 but this illustrates the idea). Then to retain the spatial information, we can perform an **averaging across depths/channels of each kernel** as such:

$$
\A_{\text{average feature maps across depth}} = \dfrac{\A^1 + \A^2}{\textbf{num_feature_maps}} = \dfrac{\A^1 + \A^2}{2} = \begin{bmatrix} 2.5 & 2 \\ 1.5 & 6.5 \end{bmatrix}
$$

Then we flatten the $\A_{gap}$ to a single vector and pass it to linear layer.

<img src="https://storage.googleapis.com/reighns/reighns_ml_projects/docs/deep_learning/computer_vision/gradcam_and_variants/feature_maps_combined.PNG" style="margin-left:auto; margin-right:auto"/>
<p style="text-align: center">
    <b>Feature Map A1 and A2 but Averaged over Channels; By Hongnan G.</b>
</p>

**A correct example**

$$
\A^1 = \begin{bmatrix} 2 & 3 \\ 1 & 5 \end{bmatrix} \quad \A^2 = \begin{bmatrix} 3 & 1 \\ 2 & 8 \end{bmatrix}
$$

Assume for a moment $\A^1$ and $\A^2$ are our feature maps (although I mentioned it was 28 by 28 but this illustrates the idea). Then to retain the spatial information, we can perform an **averaging across each kernel** as such:

$$
\text{mean}\left(\A^1\right) = \dfrac{2+3+1+5}{2 \times 2} = 2.75 \quad \text{mean}\left(\A^2\right) = \dfrac{3+1+2+8}{2 \times 2} = 3.5
$$

where by $2 \times 2$ means the number of pixels in the feature map;

and thus the

$$
\A_{\text{gap}} = \begin{bmatrix}2.75 \\ 3.5 \end{bmatrix}
$$


The Grad-CAM paper also applied the **GAP** idea over each feature map $\A_k$. One can refer to [here](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/) to understand the intuition here. 

Basically each unique feature map will be reduced to a single value, for eg, $2.75$, which *summarizes* the entire feature map.

## The Intuition

> This part is referenced from **Interpretable Machine Learning by Christoph Molnar**. He started off with an intuitive description of Grad-CAM.

- **High level idea:** Grad-CAM is to understand at which parts of an image a convolutional layer "looks" for a certain classification.
- As a reminder, the first convolutional layer of a CNN takes as input the images and outputs feature maps that encode learned features (see the chapter on [Learned Features](#cnn-features)).
    - The higher-level convolutional layers do the same, but take as input the feature maps of the previous convolutional layers.
- To understand how the CNN makes decisions, Grad-CAM analyzes which regions are activated in the feature maps of the last convolutional layers.
- There are $k$ feature maps in the last convolutional layer, and I will call them $\A^1, \A^2, \ldots, \A^k$.
- How can we "see" from the feature maps how the convolutional neural network has made a certain classification?
    - In the first approach, we could simply visualize the raw values of each feature map, average over the feature maps and overlay this over our image. This is the method of visualization feature map activations.
    - This would not be helpful since the feature maps encode information for **all classes**, but we are interested in a particular class.
    - Grad-CAM has to decide how important each of the k feature map was to our class c that we are interested in.
- We have to weight each pixel of each feature map with the gradient before we average over the feature maps.
- This gives us a heatmap which highlights regions that positively or negatively affect the class of interest.
- This heatmap is send through the `ReLU` function, which is a fancy way of saying that we set all negative values to zero.
    - Grad-CAM removes all negative values by using a ReLU function, with the argument that we are only interested in the parts that contribute to the selected class c and not to other classes.
- The word pixel might be misleading here as the feature map is smaller than the image (because of the pooling units) but is mapped back to the original image.
- We then scale the Grad-CAM map to the interval [0,1] for visualization purposes and overlay it over the original image.

## The Big Picture

This part can be best understood alongside my drawings.

The big picture is when you "reduce" the 3 feature maps into 1 single new "feature map" (localization map) by way of a linear sum where each feature map $\A^k$ is multiplied by the global average pooled gradients $\alpha_k^c$. In the example on my drawings, it can be seen that feature maps $\A^1$ and $\A^2$ are more important for the model to distinguish the elephant class than that of $\A^3$. 

By design, I made $\A^3$ in itself already look different in terms of values from $\A^1$ and $\A^2$ on the get go. But at this point in time, even though we know the values in feature map $\A^3$ are vastly different, we cannot say for sure whether $\A^3$ in itself contributed to the prediction of our class elephant or not. It could also be the case that $\A^1$ and $\A^2$ are bonkers. Thus, the concept of taking the **gradient** of the class $\y^c$ with respect to each of these feature maps $\A^k$ becomes increasingly important as now we have a mathematical way to assign some importance to each of these feature maps.

Now to even further bring out the intuition, imagine the 3 feature maps for the input image elephant as follows:

- Feature map $\A^1$ corresponds to Sobel Filter where it detect edges in particular their **trunk** of the **elephants**.
- Feature map $\A^2$ corresponds to Another Filter where it detects the **ear** of the **elephants**.
- Feature map $\A^3$ corresponds to a Background Filter where it detect the background and in particular the grass patch under the **elephants**.

---

Now this is apparent why $\A^3$ is vastly different in terms of values since it is actually representing the background and grass. We know that grass is a non-factor in determining an elephant since the african grass field can hold many other animals as well. We really do not want the model to assign important to the grass, if any at all. 

We can then compute the gradient of $\y^c$ wrt each feature map and we can intuitively understand it as (see my diagrams):

- Gradient map $\frac{d\y^{386}}{d\A^1}$ has values $\begin{bmatrix} 2 & 3 \\ 4 & 2 \end{bmatrix}$
    - These values are the gradients of feature map $\A^1$ at a pixel level. We may find it difficult to comprehend these gradients and we want a quick summary of how feature map $\A^1$ changes our model's decision for $\y^{386}$. Thus we use GAP.
    - and when we perform GAP on them, we have $\alpha_{1}^{386} = 2.75$. 
    - This value roughly translates to a pertubation of $2.75$ in feature map $\A^1$ will result in a unit change in our class $\y^{386}$.
- Gradient map $\frac{d\y^{386}}{d\A^2}$ has values $\begin{bmatrix} 3 & 5 \\ 6 & 3 \end{bmatrix}$ and when we perform GAP on them, we have $4.25$. 
    - The same logic applies.
- Gradient map $\frac{d\y^{386}}{d\A^3}$ has values $\begin{bmatrix} 0.1 & -0.1 \\ 0.2 & 0.2 \end{bmatrix}$ and when we perform GAP on them, we have $0.1$.
    - The same logic applies and we see that the gradient is small here and hence we can deduce that this feature map is not very important.

---




Ultimately however, we want to know how the change in **all feature maps $\A^k$** affects our final decision. This is where we perform a linear sum of $\left(\sum_{k} \alpha_k^c A^k\right)$ of all feature maps with the weight coefficient being their global average pooled gradient map. Just like our good old linear regression model, the intuition is the same as in $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ...$. The coefficients (the gradient values of $2.75, 4.25, 0.1$ decides which feature map are important and gets "activated and highlighted").

- We get the weighted sum heatmap to be the shape $(2, 2)$ of values $[8.925, -2.025, 16.125, 2.6]$. Now this is a very small map and is likely not going to happen in real life. More often the map at this stage is of size $(10, 10)$ and beyond. Imagine once more how $\A^3$ played almost no role.

---

Lastly, an elementwise `ReLU` operation is applied to the heatmap to get $[8,925, 0, 16.125, 2.6]$ and from the paper: We apply a `ReLU` to the linear combination of maps because we are only interested in the features that have a positive influence on the class of interest, i.e. pixels whose intensity should be increased in order to increase $y^{c}$. Intuitively, the idea is that negative values contribute to other classes and we should not really care about it.

We can scale back the heatmap to overlay on the original image.

From [here](https://medium.com/@stepanulyanin/implementing-grad-cam-in-pytorch-ea0937c31e82), author said:

> Another potential question that can arise is why wouldn’t we just compute the gradient of the class logit with respect to the input image. Remember that a convolutional neural network works as a feature extractor and deeper layers of the network operate in increasingly abstract spaces. We want to see which of the features actually influenced the model’s choice of the class rather than just individual image pixels. That is why it is crucial to take the activation maps of deeper convolutional layers.

## The Algorithm

With reference to [Interpretable Machine Learning by Christoph Molnar](https://christophm.github.io/interpretable-ml-book/pixel-attribution.html).

### Notation

- Input image: $\X$

- **Feature Maps** of the last convolutional layer in a CNN is denoted as:

    $$\A_1, \A_2, \ldots, \A_k$$
    
    The $k$ is an arbitrary number for the number of feature maps.

- The **Feature Logits** of a particular **class $c$** is denoted as:
    
    $$\y^{c}$$
    
    In other words, in **ImageNet**, the elephant class is indexed $386$, and is denoted $\y^{386}$. This is also the raw feature logits output before the softmax layer.
    
- The **gradient of the fully-connected logits for class $c$, $\y^{c}$ (before the softmax), with respect to feature map activations $\A_k$ of a convolutional layer**:
    
    $$\dfrac{d\y^{c}}{d\A^{k}}$$
    
- It follows that the **gradient of the fully-connected for each class $c$, $\y^{c}$, with respect to each pixel on the feature map activations $\A_k$ is denoted as**:
    
    $$\dfrac{d\y^{c}}{d\A^{k}_{ij}}$$
    
- Let us define the gradient of $\y^c$ with respect to the GAP of each feature map $\A^k$ to be:

    $$\alpha_k^c = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\delta y^c}{\delta A_{ij}^k}}_{\text{gradients via backprop}}$$

- Denote the `ReLU` function as:
    
    $$\textbf{ReLU}$$

- The final heatmap, also called the localization map of Grad-CAM is denoted as:

    $$\L^{c}_{Grad-CAM} \in \R^{w \times h}$$
    
    where $w$ and $h$ is the width and height of the final output localization map.
    
    and is equals to
    
    $$\L^{c}_{Grad-CAM} \in \R^{w \times h} = \underbrace{ReLU}_{\text{Pick positive values}}\left(\sum_{k} \alpha_k^c A^k\right)$$

### The Algorithm

We will only assume our input is 1 single image.

<!-- Pseudo code-->
Let us look at the recipe for Grad-CAM.
Our goal is to find the localization map, which is defined as:

$$L^c_{Grad-CAM} \in \mathbb{R}^{u\times v} = \underbrace{ReLU}_{\text{Pick positive values}}\left(\sum_{k} \alpha_k^c A^k\right)$$


Here, u is the width, v the height of the explanation and c the class of interest.

1. Forward-propagate the input image through the convolutional neural network.
1. Obtain the raw score for the class of interest, meaning the activation of the neuron before the softmax layer.
1. Set all other class activations to zero.
1. Back-propagate the gradient of the class of interest to the last convolutional layer before the fully connected layers: $\frac{\delta{}y^c}{\delta{}A^k}$.
1. Weight each feature map "pixel" by the gradient for the class. Indices i and j refer to the width and height dimensions:
   $$\alpha_k^c = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\delta y^c}{\delta A_{ij}^k}}_{\text{gradients via backprop}}$$
This means that the gradients are globally pooled.
1. Calculate an average of the feature maps, weighted per pixel by the gradient.
1. Apply ReLU to the averaged feature map.
1. For visualization: Scale values to the interval between 0 and 1. Upscale the image and overlay it over the original image.
1. Additional step for Guided Grad-CAM: Multiply heatmap with guided backpropagation.


#### Step 1: Forward-propagate

This step just takes in **one single image** $\mathcal{X}$ and perform a forward pass the input image through the convolutional neural network and save **all the forward pass feature map activations**.

```python
# the y_logits before softmax; forward pass to populate the forward_activations
y_logits = model(image)
# this dict will be populated with the feature map outputs
forward_activations = {"features.11_ReLU(inplace=True)": feature_map_logits}
```

---

We now have the feature maps stored and we can get our target conv layer.

$$
\begin{bmatrix} \A^1 & \A^2 & \A^3 & \cdots & \A^k \end{bmatrix}
$$

where each $\A^i \in \R^{f \times f}$.

#### Step 2: Backward-propagate

In Grad-CAM, we need the gradients of the **target category** with respect to the **target convolutional layer**. That is to say, when we call backwards, we are only interested in the particular class's gradients wrt feature maps.

```python
if target_category is None:
    # get the most likely prediction of the model
    target_category = model(image).argmax(dim=1)
    # convert to scalar
    target_category = target_category.item()

# call backward on the model to populate backward_gradients
y_logits[:, target_category].backward()

# this dict will be populated with the gradients
backward_gradients = {"features.11_ReLU(inplace=True)": gradients_yc_wrt_features.11}
```

The above code just says, if we did not choose a **target category**, the model will choose the one with the highest logit activation. We then call `y_logits[: target_category]` backwards to store the gradients in `backward_gradients`.

We now have the gradients of the class of interest with respect to the target conv layer (usually last conv layer).

$$
\begin{bmatrix} \dfrac{d\y^{c}}{d\A^{1}} & \dfrac{d\y^{c}}{d\A^{2}} & \dfrac{d\y^{c}}{d\A^{3}} & \cdots & \dfrac{d\y^{c}}{d\A^{k}} \end{bmatrix}
$$

where each $\dfrac{d\y^{c}}{d\A^{i}} \in \R^{f \times f}$.

#### Step 3: Global Average Pool Gradients

The intuition we explained earlier tells us that we need to **assign an importance score** to each **feature map $\A^i$**. How to do that? In my naive thought, since $\dfrac{d\y^{c}}{d\A^{i}}$ is the same shape as the feature map $f \times f$, I would have thought we can just multiply them elementwise. More concretely,

If

$$
\A^1 = \begin{bmatrix} 2 & 3 \\ 1 & 5 \end{bmatrix} \quad \A^2 = \begin{bmatrix} 3 & 1 \\ 2 & 8 \end{bmatrix}
$$

and

$$
\dfrac{d\y^{c}}{d\A^{1}} = \begin{bmatrix} 0.1 & 0.2 \\ 0.3 & 0.5 \end{bmatrix} \quad \dfrac{d\y^{c}}{d\A^{2}} = \begin{bmatrix} 0.3 & 0.1 \\ 0.2 & 0.3 \end{bmatrix}
$$

Then we can weigh each feature map on a pixel level:

$$
\A^1 * \dfrac{d\y^{c}}{d\A^{1}} = \begin{bmatrix} 0.2 & 0.6 \\ 0.3 & 0.25 \end{bmatrix} \quad \A^2 * \dfrac{d\y^{c}}{d\A^{2}} = \begin{bmatrix} 0.9 & 0.1 \\ 0.4 & 0.24 \end{bmatrix}
$$

---

However, the paper suggested that we can perform a **Global Average Pooling** on the gradient maps first, the idea is the same, we assume that the average of each gradient map should be representative of the rate of change of $y^c$ with respect to each feature map.

$$\alpha_k^c = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\delta y^c}{\delta A_{ij}^k}}_{\text{gradients via backprop}}$$

where $Z$ is the total number of pixels in this gradient map.

Applying this to our example:

$$
\alpha_1^c = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\delta y^c}{\delta A_{ij}^1}}_{\text{gradients via backprop}} = \frac{1}{4}\left(0.2+0.6+0.3+0.25\right) = 0.3375
$$

$$
\alpha_2^c = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\delta y^c}{\delta A_{ij}^2}}_{\text{gradients via backprop}} = \frac{1}{4}\left(0.9+0.1+0.4+0.24\right) = 0.41
$$

These $\alpha_k^c$ will be the **importance score of each feature map $\A^k$**. We will stack them into vectors:

$$
\begin{bmatrix} \alpha_1^c & \alpha_2^c & \alpha_3^c & \cdots & \alpha_k^c \end{bmatrix}
$$

and $\alpha_k^c \in \R$ is a scalar.

---

In code it is of the form:

```python
dyc_dA = backward_gradients[target_layer_name]

# average pool the gradients across the channels
global_average_pooled_gradients = torch.mean(dyc_dA, dim=[0, 2, 3])
```

#### Step 4: Weighted Global Sum: Localized Feature Maps

Recall that good old linear regression has the form $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_k x_k$. Now we do the same here and treat $\alpha_k^c$ as the beta weight coefficients, and feature maps $\A^k$ as our features to get:

$$\textbf{Localized_Feature_Map} = \left(\sum_{k} \alpha_k^c A^k\right)$$

which has a shape of $\R^{f \times f}$. The intuition is that some unimportant feature maps will go down to $0$ or near $0$, and the important feature maps are magnified!

---

The code has this structure:

```python
weighted_localization_map = torch.clone(forward_conv_activations)

# weight the channels by corresponding gradients
for i in range(num_feature_maps):
    weighted_localization_map[
        :, i, :, :
    ] *= global_average_pooled_gradients[i]

# sum the channels of the activations
weighted_localization_map = torch.sum(
    weighted_localization_map, dim=1
).squeeze()
```

#### Step 5: Grad-CAM Localization Map with ReLU

The last step is to apply `ReLU` to the $\textbf{Localized_Feature_Map}$ above. This is really to zero out all negative entries. Why? Well, for one, in typical logistic regression, negative weights are associated with the model placing importance in the "other class of interest", and we do not want that. The same analogy can be applied here though I believe mine is hand-wavy at best.

$$L^c_{Grad-CAM} \in \mathbb{R}^{f \times f} = \underbrace{ReLU}_{\text{Pick positive values}}\left(\sum_{k} \alpha_k^c A^k\right)$$

---

The code has structure like:

```python
# relu on top of the heatmap expression (2) in https://arxiv.org/pdf/1610.02391.pdf
relu_weighted_localization_map = torch.nn.ReLU(inplace=False)(
    weighted_localization_map
)
# normalize the heatmap, scale features to between 0 and 1 to plot
gradcam_heatmap = relu_weighted_localization_map / torch.max(
    relu_weighted_localization_map
)
```

#### Step 6: Superimpose the Grad-CAM Heatmap to Original Image

This step is just doing some resizing from the $f \times f$ to say $224 \times 224$ to overlay on the original image. 

## Dissecting KERAS's code

https://keras.io/examples/vision/grad_cam/ has sample code but I converted to PyTorch, before converting, I annotated each of the variable inside. (See my KERAS notebook as I changed some variables name).

- `last_conv_layer_output`
    - Shape: $2 \times 2 \times 3$ with 3 filters of 2 by 2 shape.
    - Note this is just $\A^1, \A^2, \A^3$ where each $\A^k \in \R^{2 \times 2}$.

- `y_logits`
    - Shape: $1 \times 1000$ since there are 1000 classes in ImageNet.
    - Note that this is our $\y$. If we are interested in the elephant class at index 386, then we denote it as $\y^{c} = \y^{386}$.
    - Thus, $\y \in \R^{1 \times 1000}$ but $\y^{c} \in \R^{1 \times 1}$.
    - Note carefully this is the logits output of **all the layers** of the CNN, and is just right before the **softmax** where we transform the logits to probabilities.
    - We can imagine that among these 1000 logits, the highest number means the model thinks that this index is the most probable class.
    - Theoretically speaking, there is no difference in differentiation of the logits wrt to the feature maps versus the softmax probs wrt to the feature maps. This is because **softmax is monotonic**, and the pre-softmax logits output will tell us already which class is most probable, and so will the softmax. So there should not be any confusion here on why we did not differentiate softmax probs wrt to the feature maps instead since the ranking is preserved in the sense that values in logits the highest is the most probable when transformed by softmax.

- `target_category`
    - This is an optional argument in the function. 
    - If `None`, we will automatically assign it to the highest logit's index.
    - In this example, the highest logit in $\y$ is at index 386 with a value of $23.632$, corresponding to the target class of elephant.
    - If specified, then the $\y^{c}$ will change.

- `target_category_logits`
    - Once our `target_category` is defined, we will just slice `y_logits[:, target_category]` to get the logit value of that particular class.
    - This variable is just $\y^{c}$ if you look carefully.

- `grads = tape.gradient(target_category_logits, last_conv_layer_output)`
    - This is the gradient of the output neuron (top predicted or chosen) with regard to the output feature map of the last conv layer.
    - This has shape $2 \times 2 \times 3$ in our simple example. In the python example, the shape at this moment has an additional axis like $1 \times 2 \times 2 \times 3$ which we will eventually squeeze it anyways.
    - Notice it has the same shape as `last_conv_layer_output` aka the feature maps.
    - In other words, this variable has 3 feature maps stacked together and can be visualized as:     
    
        $$\begin{bmatrix} \dfrac{d\y^{c}}{d\A^{1}} & \dfrac{d\y^{c}}{d\A^{2}} & \dfrac{d\y^{c}}{d\A^{3}} \end{bmatrix}$$

- `pooled_grads`
    - Shape: $3 \times 1$
    - This is a vector where each entry is the mean intensity of the gradient over a specific feature map channel.
    - In other words, in `grads[0]` we have $\frac{d\y^c}{d\A^1}$ which is of shape $2 \times 2$. What we do not is **Global Average Pooling** on these gradients where we just take the average of all available pixels in this $2 \times 2$ gradient map for feature map 1. You can find more intuition of GAP above, but in general this is similar logic to how you perform GAP on the output of the feature logits from the last conv layer.
    
        $$\alpha_k^c = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\delta y^c}{\delta A_{ij}^k}}_{\text{gradients via backprop}}$$
        
        where $Z$ is the total number of pixels in this gradient map.

- `heatmap`
    - Shape: $2 \times 2$ which is also the filter size and also the feature map size.
    - This is the weighted sum of all feature maps $\A^1, \A^2, \A^3$ and is
        
        $$\textbf{Localized_Map} = \left(\sum_{k} \alpha_k^c A^k\right)$$